# 第 3 天 - 对话式 AI - 即聊天机器人！

In [ ]:
# 导入库

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# 加载名为 .env 文件中的环境变量
# 打印密钥前缀以帮助调试

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API 密钥存在，前缀为 {openai_api_key[:8]}")
else:
    print("未设置 OpenAI API 密钥")
    
if anthropic_api_key:
    print(f"Anthropic API 密钥存在，前缀为 {anthropic_api_key[:7]}")
else:
    print("未设置 Anthropic API 密钥")

if google_api_key:
    print(f"Google API 密钥存在，前缀为 {google_api_key[:8]}")
else:
    print("未设置 Google API 密钥")

In [ ]:
# 初始化

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = "你是一个乐于助人的助手"

# 请阅读！
Gradio 已经升级了！现在它会以完全符合 OpenAI 的格式传入 `history`，非常适合我们直接发送给 OpenAI。

新版本中，开发者可以在创建 gr.ChatInterface 时通过设置参数 type="messages"，让传递给函数的 history 直接就是 OpenAI API 所需的“消息字典列表”格式，形如 [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]。

我们将编写一个函数 `chat(message, history)`，其中：
**message** 是要使用的提示
**history** 是过去的对话，采用 OpenAI 格式

我们将结合系统消息、历史记录和最新消息，然后调用 OpenAI。

In [ ]:
# 我们可以轻松地创建这个调用 OpenAI 的函数
# 现在准备 OpenAI 的输入只需要 1 行代码！

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("history是：")
    print(history)
    print("而 messages 是：")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## 接着进入 Gradio 的魔法世界！

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message = "你是一家服装店里乐于助人的店员。你应该尝试温和地鼓励 \
顾客试穿正在打折的商品。帽子六折，其他大部分商品五折。 \
例如，如果顾客说‘我想买一顶帽子’， \
你可以回复类似‘太好了 - 我们有很多帽子 - 其中有几款正在参加我们的促销活动。’\
如果顾客不确定要买什么，鼓励他们买帽子。"

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\n如果顾客问起鞋子，你应该回答说鞋子今天不打折， \
但要提醒顾客看看帽子！"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:

# 我还改进了此函数的结构

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message or '皮带' in message:
        relevant_system_message += " 本店不卖皮带；如果被问到皮带，请务必指出其他在售商品。"
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">商业应用</h2>
            <span style="color:#181;">对话式助手当然是生成式 AI 一个非常常见的用例，而最新的前沿模型在细致入微的对话方面表现得非常出色。Gradio 使得拥有用户界面变得容易。我们涵盖的另一个关键技能是如何使用提示来提供上下文、信息和示例。
<br/><br/>
考虑如何将 AI 助手应用于您的业务，并为自己制作一个原型。使用系统提示为您的业务提供背景，并为 LLM 设定基调。</span>
        </td>
    </tr>
</table>